In [1]:
try:
    __IPYTHON__
    USING_IPYTHON = True
    %load_ext autoreload
    %autoreload 2
except NameError:
    USING_IPYTHON = False

#### Argparse

In [2]:
import argparse
ap = argparse.ArgumentParser()
ap.add_argument('project_root', help='')
ap.add_argument('--mrp-data-dir', default='data', help='')
ap.add_argument('--graphviz-sub-dir', default='visualization/graphviz', help='')
ap.add_argument('--train-sub-dir', default='training', help='')
ap.add_argument('--companion-sub-dir', default='companion')

ap.add_argument('--mrp-file-extension', default='.mrp')
ap.add_argument('--companion-file-extension', default='.conllu')
ap.add_argument('--graphviz-file-template', default='http://localhost:8000/files/proj29_ds1/home/slai/mrp2019/visualization/graphviz/{}/{}.mrp/{}.png')
arg_string = """
    /data/proj29_ds1/home/slai/mrp2019
"""
arguments = [arg for arg_line in arg_string.split(r'\\n') for arg in arg_line.split()]

In [3]:
if USING_IPYTHON:
    args = ap.parse_args(arguments)
else:
    args = ap.parse_args()

In [4]:
args

Namespace(companion_file_extension='.conllu', companion_sub_dir='companion', graphviz_file_template='http://localhost:8000/files/proj29_ds1/home/slai/mrp2019/visualization/graphviz/{}/{}.mrp/{}.png', graphviz_sub_dir='visualization/graphviz', mrp_data_dir='data', mrp_file_extension='.mrp', project_root='/data/proj29_ds1/home/slai/mrp2019', train_sub_dir='training')

#### Library imports

In [5]:
import json
import logging
import os
import pprint
import string

from collections import Counter
from collections import defaultdict
from tqdm import tqdm
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import plot_util

#### ipython notebook specific imports

In [6]:
if USING_IPYTHON:
    # matplotlib config
    %matplotlib inline

In [7]:
sh = logging.StreamHandler()
formatter = logging.Formatter('%(name)s - %(levelname)s - %(message)s')
sh.setFormatter(formatter)
logging.basicConfig(level=logging.INFO, handlers=[sh])
logger = logging.getLogger(__name__)  # pylint: disable=invalid-name
logger.setLevel(logging.INFO)

### Constants

In [8]:
UNKWOWN = 'UNKWOWN'

### Load data

In [23]:
from preprocessing import MrpDataset

In [24]:
train_dir = os.path.join(args.project_root, args.mrp_data_dir, args.train_sub_dir)

In [25]:
mrp_dataset = MrpDataset()

In [26]:
frameworks, framework2dataset2mrp_jsons = mrp_dataset.load_mrp_json_dir(
    train_dir, args.mrp_file_extension)

frameworks: 100%|██████████| 5/5 [00:16<00:00,  2.92s/it]t/s]


In [13]:
for framework in framework2dataset2mrp_jsons:
    logger.info(framework)
    logger.info(list(framework2dataset2mrp_jsons[framework].keys()))

__main__ - INFO - ucca
__main__ - INFO - ['wiki', 'ewt']
__main__ - INFO - psd
__main__ - INFO - ['wsj']
__main__ - INFO - eds
__main__ - INFO - ['wsj']
__main__ - INFO - dm
__main__ - INFO - ['wsj']
__main__ - INFO - amr
__main__ - INFO - ['xinhua', 'wsj', 'wiki', 'wb', 'rte', 'proxy', 'mt09sdl', 'lorelei', 'fables', 'dfb', 'dfa', 'cctv', 'bolt', 'amr-guidelines']


### Data Preprocessing companion

In [14]:
from preprocessing import CompanionParseDataset

In [15]:
companion_dir = os.path.join(args.project_root, args.mrp_data_dir, args.companion_sub_dir)

In [16]:
cparse_dataset = CompanionParseDataset()

In [19]:
dataset2cid2parse = cparse_dataset.load_companion_parse_dir(companion_dir, args.companion_file_extension)

preprocessing - INFO - framework amr found
dataset: 100%|██████████| 13/13 [00:01<00:00,  9.60it/s]
preprocessing - INFO - framework dm found
dataset: 100%|██████████| 5/5 [00:05<00:00,  1.49s/it]
preprocessing - INFO - framework ucca found
dataset: 100%|██████████| 6/6 [00:00<00:00, 27.53it/s]


In [20]:
dataset2cid2parse.keys()

dict_keys(['amr-guidelines', 'bolt', 'cctv', 'dfa', 'dfb', 'fables', 'lorelei', 'mt09sdl', 'proxy', 'rte', 'wb', 'wiki', 'xinhua', 'wsj', 'ewt'])

In [21]:
# Some data is missing
'20003001' in dataset2cid2parse['wsj']

False

### Plot companion data

In [27]:
dataset = 'wiki'
dataset = 'wsj'
framework = 'dm'
framework = 'eds'
framework = 'psd'
framework = 'ucca'
framework = 'amr'
mrp_index = 26

parse = None
while not parse:
    mrp_index += 1
    cid = framework2dataset2mrp_jsons[framework][dataset][mrp_index]['id']
    parse = dataset2cid2parse[dataset].get(cid)
    
# print sentence
print(mrp_index, ' '.join([word_record[1] for word_record in parse]))

# print corresponding mrp png link
for framework in frameworks:
    if dataset not in framework2dataset2mrp_jsons[framework]:
        continue
    if len(framework2dataset2mrp_jsons[framework][dataset]) <= mrp_index:
        continue
    if framework2dataset2mrp_jsons[framework][dataset][mrp_index]['id'] == cid:
        print(args.graphviz_file_template.format(framework, dataset, cid))

# 
def plot_parse(parse, args, dataset, cid):
    """plot parse and print png filename"""
    parse_dg = plot_util.parse_to_directed_graph(parse)
    dataset_dir = os.path.join(args.graphviz_sub_dir, dataset)
    image_path = os.path.join(dataset_dir, cid + '.png')
    os.makedirs(dataset_dir, exist_ok=True)
    plot_util.directed_graph_to_graphviz_image(parse_dg, image_path)
    return os.path.join('http://localhost:8000/files/proj29_ds1/home/slai/mrp2019', image_path)
    
parse_plot_filename = plot_parse(parse, args, dataset, cid)
print(parse_plot_filename)

28 “ But you have to recognize that these events took place 35 years ago .
http://localhost:8000/files/proj29_ds1/home/slai/mrp2019/visualization/graphviz/psd/wsj.mrp/20003029.png
http://localhost:8000/files/proj29_ds1/home/slai/mrp2019/visualization/graphviz/eds/wsj.mrp/20003029.png
http://localhost:8000/files/proj29_ds1/home/slai/mrp2019/visualization/graphviz/dm/wsj.mrp/20003029.png
http://localhost:8000/files/proj29_ds1/home/slai/mrp2019/visualization/graphviz/amr/wsj.mrp/20003029.png
http://localhost:8000/files/proj29_ds1/home/slai/mrp2019/visualization/graphviz/wsj/20003029.png
